<a href="https://youtu.be/A2euuevpYis" target="_parent"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/YouTube_full-color_icon_%282017%29.svg/71px-YouTube_full-color_icon_%282017%29.svg.png" alt="Open In Colab"/></a>


# **Práctico Sistemas Recomendadores: pyreclab - Slope One**

En este práctico seguiremos utilizando [pyreclab](https://github.com/gasevi/pyreclab), con el cual estamos aprendiendo distintas técnicas de recomendación. Seguiremos usando la misma base de datos de los prácticos anteriores, para que puedan comparar los métodos y sus implementaciones. Este práctico está acompañado de un [video comentando la actividad](https://youtu.be/A2euuevpYis).

En esta oportunidad exploraremos el recomendador de Pendiente Uno o **Slope One** [1].

**Adaptado y preparado por:** Francisca Cattan 📩 fpcattan@uc.cl

Referencias 📖
------
[1] *Lemire, D., & Maclachlan, A. (2005, April). Slope One Predictors for Online Rating-Based Collaborative Filtering. In SDM (Vol. 5, pp. 1-5).*


**Nombre**:  Matías Ramos

## Actividad 1 👓

Antes de empezar con el práctico, responde la siguiente pregunta con lo visto en clases.

**Pregunta:** Explique cómo funciona Slope One (como modelo teórico, no piense en la implementación). En particular explique:

- Repasemos: ¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo?
- Este Filtrado Colaborativo, ¿está basado en el usuario o en los items? ¿Por qué?
- ¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)
- ¿Qué pasaría si se agrega un nuevo rating a la base de datos?
- Opcional: ¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings?

💡 *Hint: La bibliografía todo lo puede.*

**Respuesta:**

Es un algoritmo de filtrado colaborativo ya que las recomendaciones que realiza se basa en información obtenida de otros usuarios, entonces el filtrado que se realiza utiliza la 'colaboración' del resto de los usuarios vía su información que en este caso corresponde a los ratings que dan a los ítems.

Slope One es item-based ya que consiste en calcular diferencias promedios entre ítems respecto los ratings que han obtenidos. Con esta diferencia se permite estimar el rating desconocido. Al realizar los cálculos basados en el vector de ratings de los ítems se cataloga como item-based.

Slope one debe recibir los ítems y sus ratings. El arreglo común para representar tal información es la matriz user-item. Slope One trabaja en perspectiva de las columnas de esta matriz que representan los ratings de un ítem en particular. Si se quiere estimar el rating que daría un usuario u a un item i, se debe calcular el promedio ponderado de las diferencias del resto de los ítems que el usuario u evaluó, con el item i para obtener la predicción.

Si un usuario agrega un nuevo rating se debe recomputar la suma ponderada de diferencias de los k ítems los cuales el usuario a evaluado previamente. Como k es un valor generalmente no alto, el agregar un nuevo rating no tiene costos significativos.

Como el recomendador se basa en los ítems los cuales el usuario a valorado previamente, el hecho de que no tenga ratings previos originaría un cold start problem. Si tiene pocos ratings probablemente la predicción no sea buena ya que se basaría en la diferencia promedio de muy pocos ítems generando resultados sub-óptimos.




# **Configuración Inicial**

## Paso 1:
Descargue directamente a Colab los archivos del dataset ejecutando las siguientes 3 celdas:


In [1]:
!curl -L -o "u1.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    641      0 --:--:-- --:--:-- --:--:--   641
100 1546k  100 1546k    0     0  1985k      0 --:--:-- --:--:-- --:--:--  107M


In [2]:
!curl -L -o "u1.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   1083      0 --:--:-- --:--:-- --:--:--  1083
100  385k  100  385k    0     0   739k      0 --:--:-- --:--:-- --:--:--  125M


In [3]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    970      0 --:--:-- --:--:-- --:--:--   970
100  230k  100  230k    0     0   385k      0 --:--:-- --:--:-- --:--:--  385k


Los archivos **u1.base** y **u1.test** tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [movielens](https://grouplens.org/datasets/movielens/).

## Paso 2:

Instalamos pyreclab utilizando pip.

In [4]:
!pip install pyreclab --upgrade

     |████████████████████████████████| 235kB 1.6MB/s 


## Paso 3:

Hacemos los imports necesarios para este práctico.

In [5]:
import pyreclab
import numpy as np
import pandas as pd

# **El dataset**

💡 *En prácticos anteriores, vimos como analizar este dataset. Puedes revisarlos en caso de dudas.*

## Paso 4:

Ya que queremos crear una lista de recomendación de items para un usuario en especifico, necesitamos obtener información adicional de cada película tal como título, fecha de lanzamiento, género, etc. Cargaremos el archivo de items descargado "u.item" para poder mapear cada identificador de ítem al conjunto de datos que lo describe.

In [6]:
# Definimos el orden de las columnas
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

# Asignamos a una variable la estructura de datos de los items
info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

# **Slope One**

## Paso 5:

Seguiremos un camino muy similar a los ejercicios de User KNN e Item KNN. Crearemos una instancia del algoritmo de recomendación y luego pasaremos a la fase de entrenamiento.

In [7]:
# Declaramos la instancia SlopeOne
mySlopeOne = pyreclab.SlopeOne(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)

In [8]:
# Y enntrenamos
mySlopeOne.train()

## Actividad 2 👓

**Pregunta:** Explique qué hace el método `train()` en este caso, dado el modelo teórico. ¿Calcula información?, ¿no hace nada?, ¿ordena los datos? 

**Respuesta:**

El método train() debería realizar los cálculos de Slope One. Por usuario, se calcula la suma ponderada del promedio de diferencias de ratings entre el item a predecir y los items evaluados previamente por 
él. Estos valores se guardan y se acceden cuando se consulta por la predicción o recomendación.

## Paso 6:

Llego la hora de predecir el rating.

In [9]:
# Esta es la predicción de rating que el usuario ID:457 otorgaría al ítem ID:37
# De esta forma podemos comparar el resultado con los prácticos anteriores
mySlopeOne.predict("457", "37")

3.2408759593963623

In [10]:
# También podemos guardar la predicción en una variable
prediction = mySlopeOne.predict("457", "37")

In [11]:
# Podemos comprobar las peliculas rankeadas por el usuario ID:457
# Que ciertamente ha participado activamente (¡156 items!)
train_file = pd.read_csv('u1.base', sep='\t', names = ['userid', 'itemid', 'rating', 'timestamp'], header=None)
train_file[train_file['userid'] == 457]

,userid,itemid,rating,timestamp
37269,457,1,4,882393244
37270,457,7,4,882393278
37271,457,9,5,882393485
37272,457,11,4,882397020
37273,457,13,3,882393883
...,...,...,...,...
37420,457,1047,2,882395964
37421,457,1119,4,882398308
37422,457,1168,5,882548761
37423,457,1210,4,882549905


In [13]:
# Y también cuáles usuarios han rankeado la pelicula ID:37
train_file[train_file['itemid'] == 37]

,userid,itemid,rating,timestamp
1302,13,37,1,882397011
14851,201,37,2,884114635
19670,268,37,3,876514002
29489,363,37,2,891498510
31084,385,37,4,880013483
32996,405,37,1,885548384
62777,773,37,3,888540352


## Actividad 3 👓

Haremos un pequeño experimento para entender mejor como funciona Slope One. Gracias al ejercicio anterior, sabemos que el usuario 457 ya ha asignado el mejor rating (5 ⭐) a las dos peliculas ID:9 e ID:1168. Comparemos.

**Pregunta:** ¿Cómo se explican estos resultados?  

**Respuesta:**

Una hipótesis que se puede elaborar es que probablemente el usuario tiende a valorar las otras películas con un rating más similar al promedio o neutro. De los pocos ratings que vimos en el dataframe anterior se vió que el usuario daba valoración 4 con mayor frecuencia mientras que justo las películas 9 y 1168 son aquellas que tienen el valor máximo que no se repite en otras peliculas de la selección anterior. Tambíen el usuario ha dado muchos ratings por lo tanto la suma ponderada considerara a otros 155 items. Entre más ítems, la suma ponderada probablemente tenderá a alguna medida central de rating y quizás es menos probable que prediga valores extremos como 1 o 5.

La diferencia en las predicciones se da porque la pelicula  9 tiene mayor un promedio ponderado mayor de diferencias de ratings entre el resto de los items vistos por el usuario. Es decir que el item 9 es más distante al resto de los ítems vistos por el usuario que el ítem 1168.

In [15]:
prediction_id9 = mySlopeOne.predict("457", "9")
prediction_id1168 = mySlopeOne.predict("457", "1168")

print('Prediction for ID:9 :', prediction_id9)
print('Prediction for ID:1168 :', prediction_id1168)

Prediction for ID:9 : 4.530702114105225
Prediction for ID:1168 : 4.166153907775879


## Paso 7:

Generaremos ahora una lista ordenada de las top-N recomendaciones, dado un usuario.



In [16]:
# Mediante el método recommend() genereremos una lista top-5 recomendaciones para el usuario ID:457
reclist_slopeone = mySlopeOne.recommend("457", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1592', '1589', '1656', '1431', '1653']


In [17]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1592                               Magic Hour, The (1998)
1589                                   Schizopolis (1996)
1656                                   Little City (1998)
1431                                  Legal Deceit (1997)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## Actividad 4 👩🏻‍💻

Genera una nueva recomendacion, modificando los hiperparametros de usuario y topN a tu elección.

**Pregunta:** ¿Ves una diferencia en la recomendación entre el nuevo usuario y el usuario ID:457?

**Respuesta:**

La recomendación es completamente distinta. Sugiere que existe personalización en las recomendaciones

In [18]:
# Escribe el nuevo codigo aqui
reclist_slopeone = mySlopeOne.recommend("456", 10)
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1472    Visitors, The (Visiteurs, Les) (1993)
1613                        Tokyo Fist (1995)
1536                     Aiqing wansui (1994)
1500                Santa with Muscles (1996)
1463                         Boys, Les (1997)
1293                          Star Kid (1997)
80                Hot Shots! Part Deux (1993)
78                          Free Willy (1993)
77                           Firm, The (1993)
76                       Carlito's Way (1993)
Name: title, dtype: object

## Actividad 5 👩🏻‍💻

Dado el usuario ID:44, cree dos listas de películas recomendadas; la primera utilizando el algoritmo Most Popular y la segunda utilizando el algoritmo Slope One.

**Pregunta:** Realice un analisis apreciativo de las similitudes y diferencias entre ambas recomendaciones.

**Respuesta:**

Puedo notar dos cosas. La primera es que las recomendaciones de Slope One son más homogéneas en cuanto a género. Se puede identificar que la mayoría de las peliculas que recomendó son de Drama. Por otro lado, Most popular recomendó múltiples géneros. Destacó accion, drama, romance, sci-fi y guerra.

El segundo punto que note a modo personal, las películas recomendadas en most popular las conozco en su mayoría mientras que las de Slope One son todas desconocidas para mí. 

Tomando ambos puntos en consideración, Slope One parece ser mucho más específico a la hora de recomendar en relación a Most Popular. Pensé que podría significar mejor rendimiento pero al testear por métricas MAP y nDCG@10 se puede apreciar que MostPopular tuvo mejor rendimiento.

In [20]:
most_popular = pyreclab.MostPopular(dataset='u1.base',
                   dlmchar=b'\t',
                   header=False,
                   usercol=0,
                   itemcol=1,
                   ratingcol=2)
most_popular.train()


In [29]:
reclist_slopeone = most_popular.recommend("44", 10)
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]

Lista de items por nombre:


,title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieid,,,,,,,,,,,,,,,,,,,,,,,
50,Star Wars (1977),01-Jan-1977,NaN,http://us.imdb.com/M/title-exact?Star%20Wars%2...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
100,Fargo (1996),14-Feb-1997,NaN,http://us.imdb.com/M/title-exact?Fargo%20(1996),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
181,Return of the Jedi (1983),14-Mar-1997,NaN,http://us.imdb.com/M/title-exact?Return%20of%2...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
286,"English Patient, The (1996)",15-Nov-1996,NaN,http://us.imdb.com/M/title-exact?English%20Pat...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
121,Independence Day (ID4) (1996),03-Jul-1996,NaN,http://us.imdb.com/M/title-exact?Independence%...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
300,Air Force One (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Air+Force+One...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
127,"Godfather, The (1972)",01-Jan-1972,NaN,"http://us.imdb.com/M/title-exact?Godfather,%20...",0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0


In [30]:
# Escribe el nuevo codigo aqui
reclist_slopeone = mySlopeOne.recommend("44", 10)
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]


Lista de items por nombre:


,title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieid,,,,,,,,,,,,,,,,,,,,,,,
1656,Little City (1998),20-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Little+City+(...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
1064,Crossfire (1947),01-Jan-1947,NaN,http://us.imdb.com/M/title-exact?Crossfire%20(...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
1643,Angel Baby (1995),10-Jan-1997,NaN,http://us.imdb.com/Title?Angel+Baby+(1995/I),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1642,Some Mother's Son (1996),27-Dec-1996,NaN,http://us.imdb.com/M/title-exact?Some%20Mother...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1625,Nightwatch (1997),22-Apr-1997,NaN,http://us.imdb.com/M/title-exact?Nightwatch%20...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1599,Someone Else's America (1995),10-May-1996,NaN,http://us.imdb.com/M/title-exact?Someone%20Els...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1512,"World of Apu, The (Apur Sansar) (1959)",05-Apr-1996,NaN,http://us.imdb.com/M/title-exact?Apur%20Sansar...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1536,Aiqing wansui (1994),22-Jul-1996,NaN,http://us.imdb.com/M/title-exact?Aiqing%20Wans...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1500,Santa with Muscles (1996),08-Nov-1996,NaN,http://us.imdb.com/M/title-exact?Santa%20with%...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
recommendList, maprec, ndcg = most_popular.testrec(input_file='u1.test',
                                          dlmchar=b'\t',
                                          header=False,
                                          usercol=0,
                                          itemcol=1,
                                          ratingcol=2,
                                          topn=10,
                                          relevance_threshold=2,
                                          includeRated=False)

print('MAP: {}\nNDCG@{}: {}'.format(maprec, 10, ndcg))

MAP: 0.2889129174597905
NDCG@10: 0.1598011823409535


In [27]:
recommendList, maprec, ndcg = mySlopeOne.testrec(input_file='u1.test',
                                          dlmchar=b'\t',
                                          header=False,
                                          usercol=0,
                                          itemcol=1,
                                          ratingcol=2,
                                          topn=10,
                                          relevance_threshold=2,
                                          includeRated=False)

print('MAP: {}\nNDCG@{}: {}'.format(maprec, 10, ndcg))

MAP: 0.017664149013738233
NDCG@10: 0.010398106088896622
